In [87]:
import PyPDF2 as p2
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# from goto import goto, label

In [88]:
#define the paths of the PDF scrapped 

path = 'F:/Environmental Baseline Data/Version 3/Data/PDF'
pdfs = os.listdir(path)
pdf_paths = [str(path) + '/' + str(x) for x in pdfs]
len(pdf_paths)

1329

# 1. Make Outlines Based on the Postal Codes 

This part of the code will generate a csv file with all the PDF nammes with the fillowing details: <br> - 'file_nos': just a number to indicate the file
<br> - 'file_names': the name of the file (Postal Code.pdf)
<br> - 'file_paths' : the path of the pdf file
<br> - 'int_postals' : the integer value of the postal code
<br> - 'chunk_nos': chunk_no based on the consectutive Postal Code
    

### Postal to Int Functions 

In [56]:
def postal_to_int(Postal):
    multiplier = 1
    odd = 1
    Postal_Int_Word = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
    
    # Chcek for the length of the Postal Code
    if not(len(Postal)==6):
        return(Error_Message)
    
    for letter in list(str.strip(Postal)):
        if odd%2 == 1:
            # Check if it is an interger or not   
            # If this is not an integer then exit
            if not (str.isnumeric(letter)):
                return(Error_Message)
            
            Postal_Int_Letter = multiplier * int(letter)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 10
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
        else:
            # Check if it is a character or not
            # If this is not a character then exit
            if not (str.isalpha(letter)):
                return(Error_Message)
            
            # ord('A') = 65
            Postal_Int_Letter = multiplier * (ord(letter) - 65)
            Postal_Int_Word = Postal_Int_Word + Postal_Int_Letter
            
            odd = odd + 1
            multiplier = multiplier * 26
            # print(letter, Postal_Int_Letter)
            # print(Postal_Int_Word)
            
    return Postal_Int_Word

In [57]:
# never used integer to Postal but I guess it would be useful 
# Also useful in validating if Postal to Int Function works 

def int_to_postal(Postal_Int_Word):
    Postal_Int_Word_Rem = Postal_Int_Word
    Postal = ""
    odd = 0
    Error_Message = "Invalid Postal Code"
    
    # Reversing the string 
    Postal = Postal[::-1] 
 
    while odd < 6:
        if odd%2 == 0:
            Postal_Int_Letter = Postal_Int_Word_Rem % 10
            Postal = Postal + str(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/10)
            odd = odd + 1

        else:
            Postal_Int_Letter = (Postal_Int_Word_Rem % 26) +65
            Postal = Postal + chr(Postal_Int_Letter)
            Postal_Int_Word_Rem = int(Postal_Int_Word_Rem/26)
            odd = odd + 1
            
    # Accounts for 7 digit Postal Codes
    if Postal_Int_Word_Rem > 0:
        return(Error_Message)
        
    return Postal[::-1]
  

In [58]:
file_nos = []
file_names = []
file_paths = []
int_postals = []
file_no = 1

for pdf_path in pdf_paths:
    # additional check to get rid of the string "Thumb"
    if 'Thum' in pdf_path:
        continue
    
    file_nos.append(file_no)
    file_name = str(pdf_path.split('/')[-1])
    file_names.append(file_name)
    file_paths.append(pdf_path)
    
    postal = file_name[:-4]
    int_postal = postal_to_int(postal)
    int_postals.append(int_postal)
    
len(file_nos)
len(file_names)
len(file_paths)
len(int_postals)

1328

1328

1328

1328

In [59]:
chunk_nos = []
chunk = 0
last_postal = int_postals[0]

for int_postal in int_postals:
    if int_postal == last_postal + 1:
        chunk_nos.append(chunk)
    else:
        chunk = chunk +1
        chunk_nos.append(chunk)
    last_postal = int_postal
    
len(chunk_nos)
chunk

1328

86

In [60]:
df_outlines = pd.DataFrame({'file_nos': file_nos, 'file_names': file_names, 'file_paths' : file_paths, 'int_postals': int_postals, 'chunk_nos': chunk_nos})
df_outlines.head(5)
df_outlines.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Outline_Present.csv")

,file_nos,file_names,file_paths,int_postals,chunk_nos
0,1,A0H8C0.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20300,1
1,1,A0H8C1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20301,1
2,1,A0H8C2.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20302,1
3,1,A0H8C3.pdf,F:/Environmental Baseline Data/Version 3/Data/...,20303,1
4,1,A0U3G1.pdf,F:/Environmental Baseline Data/Version 3/Data/...,52841,2


# 2. Extract Chapters (Outline level < 4) from all PDFs 

In [61]:
# Functtion that runs recursively to extract the outlines of the PDF files
def nested_is_instance_check(s, level, file_name, file_names_ch, chapters, levels, pages):
    level +=1
    if isinstance(s, (int, list, float, complex)):
        for x in s:
            nested_is_instance_check(x, level, file_name, file_names_ch, chapters, levels, pages)
    else:
        chapters.extend([s.title])
        levels.extend([level])
        #it was observed that for a lot of PDF files we could not find the page number 
        # Hence, -999 is the error code added for those cases 
        try:
            pages.extend([str(pdfread.getDestinationPageNumber(s))])
        except: 
            pages.extend([-999])
        file_names_ch.append([file_name])

In [62]:
def count_occorances(lst, x):
    count = 0
    for ele in lst:
        if ele == x:
            count = count + 1
    return count

In [63]:
current_df_outlines = df_outlines[df_outlines.chunk_nos == 3]
type(current_df_outlines.file_paths)

pandas.core.series.Series

In [75]:
outline_present = []
outline_not_present_but_TOC_present = []
file_names = []

# Variables for Initialization of the data frame for Index 4 - Chapters.csv
file_names_ch = []
chapters = []
levels = []
pages = []
all_df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})


for current_chunk in range(0, chunk+1-80):
    current_df_outlines = df_outlines[df_outlines.chunk_nos == current_chunk]
    
    # Variables for Index 4 - Chapters.csv for each Chunk
    file_names_ch = []
    chapters = []
    levels = []
    pages = []
    
    
    last_file_names_ch = []
    last_chapters = []
    last_levels = []
    last_pages = []
    
    # Going Chunk by Chunk 
    # We define chunk as all pdfs that have consecutive Postal Codes 
    for pdf_path in current_df_outlines.file_paths:
        
        # Variables for Index 4 - Chapters.csv for each Chunk
        p_file_names_ch = []
        p_chapters = []
        p_levels = []
        p_pages = []
               
        #Initializations are done here 
        file_name = str(pdf_path.split('/')[-1])
        file_names.append(file_name)
        outline_p = 0
        
        ### Try getting the outline first 
        try:
            pdfread = p2.PdfFileReader(pdf_path)
            s = pdfread.outlines
            len_s = len(s)
        except:
            len_s = -99
        
        if len_s > 0:
            level = 0
            outline_p = 1
            outline_present.append(1) # 1 Indicating that the outline is present 
            outline_not_present_but_TOC_present.append(0)
            nested_is_instance_check(s, level, file_name, p_file_names_ch, p_chapters, p_levels, p_pages)
        elif len_s == 0:
            outline_p = 0
            outline_present.append(0)
        else:
            outline_p = -1
            outline_present.append(-1)
    
              
        ## If Outline is present check if the last pdf had the same Outline
        if outline_p > 0:
            if count_occorances(pages, -999) > 1:
            ###################################################
            ############ Write a fn  to look for the same stuff in other pdf files. 
            ############
        
                # look if other PDFs have outlines too
                if p_chapters == last_chapters:
                    print("Repeatition confirmed")
                    ### Retain the chapter names from last chapter and keep moving 
                    chapters_might_modify = 1
                    
                ###################################################
                ############ Write a fn  to make it (page_nos) consistent
                ############
                
                else:
                    print("This file {} might carry on to other PDFs".format(file_name))
                    
            else:
                file_names_ch.extend(p_file_names_ch)
                chapters.extend(p_chapters)
                levels.extend(p_levels)
                pages.extend(p_pages)
                last_chapters = []
                chapters_might_modify = 0
                
                    
                    
                    
           
                    
        ## If outline is not present then let's see if theer are other ways to check             
        else:
            ###################################################
            ############ Use Other methods to extract the TOC 
            ### 1. Check in the chunk for help 
            ### 2. Check for PDF to HTMl Method to extract Table of contents 
            # if TOC Present then outline_not_present_but_TOC_present.append(1)
            
            outline_not_present_but_TOC_present.append(1)
                
                
                
        if switch_last_chapter == 1:
            last_chapters = p_chapters

            
            
                
        # Error Code for s (=pdfraed.outlines) is -99
        # the negative number allows exclusivity from the if and the elif conditions above
        # so the previous value is not affecting the next itteration in case the try block 
        # does not complete all its operations 
        len_s = -99
        switch_last_chapter = 1
            
            
    # change this based on the above mentioned if statements 
    df_chapters = pd.DataFrame({'file_names': file_names_ch, 'chapters': chapters, 'pages' : pages, 'levels' : levels})
    all_df_chapters = pd.concat([all_df_chapters, df_chapters])
    
        
        


This file A0H8C0.pdf might carry on to other PDFs
Probably Repeated Outline found at file A0H8C1.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1T0F5.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1T0F7.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1T0F8.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1T0G0.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1T0G1.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1Y9H5.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1Y9H6.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1Y9H7.pdf
Repeatition confirmed
Probably Repeated Outline found at file A1Y9H8.pdf
Repeatition confirmed
Probably Repeated Outline found at file A2F4L2.pdf
Repeatition confirmed
Probably Repeated Outline found at file A2F4L3.pdf
Repeatition confirmed
Probably Repeated Outline found at file A2F4L4.pdf
Repeatition confirmed
P

In [76]:
len(outline_present)
len(file_names)
len(df_outlines['file_names'])

1328

1328

1328

In [73]:
df_outlines['outline_present'] = outline_present

ValueError: Length of values does not match length of index

In [ ]:
##########################################################################################
########## We might need to think of a better way to extract important chapters ##########
##########  df.chapters.levels < 5, is not teh best way ##################################
##########################################################################################
all_df_chapters = all_df_chapters[all_df_chapters.chapters is not None  and all_df_chapters.levels < 5]
all_df_chapters.head(5)
all_df_chapters.to_csv("F:/Environmental Baseline Data/Version 3/Indices/Index 4 (Outlines)- Chapters.csv")

  
 

In [85]:
arra = [1, 2, 3]
arr = [1, 2, 3]
if arr == arra:
    print("yes")
arra.extend(arr)
arra

yes


[1, 2, 3, 1, 2, 3]